In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_only import*

[03:14:22] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[03:14:23] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
cnn_dropout = 0.05
batch_size = 128
bucket_num, bucket_ratio = 2, 0

In [ ]:
net = ILNet(dropout=cnn_dropout)
net.encoder = model.encoder
net.src_embed =  model.src_embed
net.hybridize()

In [4]:
import pandas as pd
TD_database = pd.read_excel(os.path.join(root_path,'datasets/il properties/TDECOMP_V.xlsx'),sheet_name='Sheet1')

In [5]:

train_smiles = TD_database['IL SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_smiles_lengths = TD_database['IL SMILES'].map(canonical_smile).map(no_split).map(get_length)
train_MP =mx.nd.array(TD_database['TDECOMP_Celcius'])
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_smiles,train_MP))
train_dataloader = get_train_dataloader(train_dataset,train_smiles_lengths,batch_size=batch_size, bucket_num=bucket_num,bucket_ratio=bucket_ratio)

In [6]:
epochs, learning_rate = 150,0.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
max_len = 100
save_name = 'TD_best.params'
train(net, train_dataloader, batch_size, learning_rate, ctx, epochs, save_name=save_name)

[03:14:28] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[03:14:29] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


[Epoch 0 Batch 10/10] avg loss 223.193, throughput 0.0394607K wps
[Epoch 0] train avg loss 223.193, train avg r2 -6.13405e+06,throughput 0.0394598K wps
learning rate: 0.001
[Epoch 1 Batch 10/10] avg loss 75.3483, throughput 0.0624678K wps
[Epoch 1] train avg loss 75.3483, train avg r2 -5.96482,throughput 0.0624659K wps
learning rate: 0.001
[Epoch 2 Batch 10/10] avg loss 65.8754, throughput 0.127358K wps
[Epoch 2] train avg loss 65.8754, train avg r2 -4.30314,throughput 0.127347K wps
learning rate: 0.001
[Epoch 3 Batch 10/10] avg loss 56.9832, throughput 0.121912K wps
[Epoch 3] train avg loss 56.9832, train avg r2 -2.11253,throughput 0.121899K wps
learning rate: 0.001
[Epoch 4 Batch 10/10] avg loss 49.1947, throughput 2.77185K wps
[Epoch 4] train avg loss 49.1947, train avg r2 -1.2218,throughput 2.76849K wps
learning rate: 0.001
[Epoch 5 Batch 10/10] avg loss 44.6037, throughput 2.93262K wps
[Epoch 5] train avg loss 44.6037, train avg r2 -0.382827,throughput 2.92935K wps
learning rate: 

(7.066577627536028, 0.9455105937460294)